In [164]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('Cars_Datasets_2025.csv', encoding='windows-1252')
df.info()

#abbiamo 10 colonne composte da oggetti

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Company Names              1218 non-null   object
 1   Cars Names                 1218 non-null   object
 2   Engines                    1218 non-null   object
 3   CC/Battery Capacity        1215 non-null   object
 4   HorsePower                 1218 non-null   object
 5   Total Speed                1218 non-null   object
 6   Performance(0 - 100 )KM/H  1212 non-null   object
 7   Cars Prices                1218 non-null   object
 8   Fuel Types                 1218 non-null   object
 9   Seats                      1218 non-null   object
 10  Torque                     1217 non-null   object
dtypes: object(11)
memory usage: 104.8+ KB


In [165]:
for col in ["Company Names","Cars Names","Engines","CC/Battery Capacity","HorsePower","Total Speed","Performance(0 - 100 )KM/H","Cars Prices","Fuel Types","Seats","Torque"]:
    group_df = df.groupby(col).count()
    print(df[col].nunique())
    print(group_df)
   
    

37
                   Cars Names  Engines  CC/Battery Capacity  HorsePower  \
Company Names                                                             
ASTON MARTIN               11       11                   11          11   
AUDI                       21       21                   21          21   
Acura                      27       27                   27          27   
BENTLEY                     1        1                    1           1   
BMW                        41       41                   41          41   
Bugatti                    10       10                   10          10   
Cadillac                   20       20                   20          20   
Chevrolet                  58       58                   58          58   
FERRARI                     9        9                    9           9   
Ford                       54       54                   54          54   
GMC                        59       59                   59          59   
HONDA                 

In [166]:
df["Company Names"] = (
df["Company Names"]
    .astype(str)
    .str.strip()
    .str.upper()
    .str.replace(r"\s+", " ", regex=True)
)

for col in ["Company Names"]:
        group_df_uniform = df.groupby(col).count()
        print(df[col].nunique())
        print(group_df_uniform)
        
#Uniformiamo le company names uguali ma scritte in modo diverso , rendendole tutte MAIUSCOLE

31
                   Cars Names  Engines  CC/Battery Capacity  HorsePower  \
Company Names                                                             
ACURA                      27       27                   27          27   
ASTON MARTIN               11       11                   11          11   
AUDI                       21       21                   21          21   
BENTLEY                     1        1                    1           1   
BMW                        41       41                   41          41   
BUGATTI                    10       10                   10          10   
CADILLAC                   20       20                   20          20   
CHEVROLET                  58       58                   58          58   
FERRARI                     9        9                    9           9   
FORD                       54       54                   54          54   
GMC                        59       59                   59          59   
HONDA                 

In [167]:
#Eliminazione colonna Engines poichè non statisticamente rilevante

engine_counts = df['Engines'].value_counts()
print("Conteggio motori:\n", engine_counts)

#Percentuale di ogni motore sul totale
engine_percentage = df['Engines'].value_counts(normalize=True) * 100
print("\nPercentuale motori sul totale:\n", engine_percentage.round(2))

#Elimino la colonna Engines dal DataFrame
df = df.drop(columns=['Engines'])
print("\nColonna 'Engines' eliminata" )



Conteggio motori:
 Engines
I4                                    64
V12                                   51
V6                                    39
Inline-4                              38
V8                                    34
                                      ..
1.5L,HYBRID                            1
2.4L Turbo I4 (i-FORCE MAX Hybrid)     1
2.0L Gas / 2.0L Hybrid                 1
1.8L / 2.0L Hybrid                     1
2.5L Hybrid / Plug-in Hybrid           1
Name: count, Length: 356, dtype: int64

Percentuale motori sul totale:
 Engines
I4                                    5.25
V12                                   4.19
V6                                    3.20
Inline-4                              3.12
V8                                    2.79
                                      ... 
1.5L,HYBRID                           0.08
2.4L Turbo I4 (i-FORCE MAX Hybrid)    0.08
2.0L Gas / 2.0L Hybrid                0.08
1.8L / 2.0L Hybrid                    0.08
2.5L Hybrid 

In [168]:
import re
import pandas as pd
import numpy as np

def extract_numeric(s):
    if pd.isna(s):
        return np.nan
    s = str(s).replace('–','-')  # normalizza trattini
    
    # Caso speciale: se contiene '+', somma i numeri
    if '+' in s:
        parts = re.findall(r'\d+(?:\.\d+)?', s)
        if not parts:
            return np.nan
        return sum(float(p) for p in parts)
    
    # Altrimenti: estrai numeri e calcola media se intervallo
    numbers = re.findall(r'\d+(?:\.\d+)?', s)
    if not numbers:
        return np.nan
    numbers = [float(n) for n in numbers]
    return np.mean(numbers)


In [169]:
import re

# Funzione per classificare Fuel Types
def normalize_fuel(x):
    if pd.isna(x):
        return np.nan
    t = str(x).lower().strip()

    # PLUG-IN HYBRID
    if re.search(r"\bplug[- ]?in\b", t) or "plug" in t:
        return "plug-in hybrid"

    # PETROL HYBRID
    if "petrol" in t and "hybrid" in t:
        return "petrol hybrid"

    # HYBRID semplice
    if "hybrid" in t:
        return "hybrid"

    # PETROL puro
    if "petrol" in t:
        return "petrol"

    # DIESEL
    if "diesel" in t:
        return "diesel"

    # ELECTRIC / EV
    if "electric" in t or "ev" in t:
        return "electric"

    # HYDROGEN
    if "hydrogen" in t:
        return "hydrogen"

    # Se nulla matcha → restituisco il testo normalizzato
    return t

# Applico la normalizzazione
df["Fuel Types"] = df["Fuel Types"].apply(normalize_fuel)

# Controllo i valori unici (per vedere se ci sono errori di battitura)
for col in ['Company Names', 'Fuel Types']:
    print(col, df[col].unique())


Company Names ['FERRARI' 'ROLLS ROYCE' 'FORD' 'MERCEDES' 'AUDI' 'BMW' 'ASTON MARTIN'
 'BENTLEY' 'LAMBORGHINI' 'TOYOTA' 'NISSAN' 'VOLVO' 'KIA' 'HONDA' 'HYUNDAI'
 'MAHINDRA' 'MARUTI SUZUKI' 'VOLKSWAGEN' 'PORSCHE' 'CADILLAC'
 'TATA MOTORS' 'TESLA' 'JEEP' 'MAZDA' 'CHEVROLET' 'GMC' 'PEUGEOT'
 'BUGATTI' 'JAGUAR LAND ROVER' 'ACURA' 'MITSUBISHI']
Fuel Types ['plug-in hybrid' 'petrol' 'diesel' 'hybrid' 'electric' 'petrol hybrid'
 'hydrogen']


In [170]:
import seaborn as sns

numeric_cols=df[['HorsePower_numeric','Total_Speed_numeric','Cars_Prices_numeric',
    'Torque_numeric','CC_Battery_numeric','Performance_numeric','Seats_numeric']]

#Funzione per calcolare IQR e individuare outlier
def find_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower) | (df[col] > upper)]
    return outliers

#Loop su tutte le colonne,
for col in numeric_cols:
    plt.figure(figsize=(8,4))
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot di {col} (outlier evidenziati)')
    plt.show()

    outliers = find_outliers(df, col)
    print(f"\nColonna: {col}")
    print(f"Numero di outlier: {len(outliers)}")
    if len(outliers) > 0:
        print(outliers[[col]])

ModuleNotFoundError: No module named 'seaborn'